# Chiffrement RSA

Tentative pour reproduire l'algorithme de chiffrement RSA d'après [wikipédia](https://fr.wikipedia.org/wiki/Chiffrement_RSA).

In [47]:
import numpy as np

In [48]:
def isprime( n ):
    if n%2==0:
        return False
    
    kmax = int( np.sqrt( n ) )+1
    for k in range(3, kmax, 2):
        if n%k==0:
            return False
        
    return True

In [49]:
print( [x for x in range(1, 200, 2) if isprime(x)] )

[1, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]


In [50]:
import random

In [51]:
def pickprime( low=101, high=999 ):
    for i in range( 100 ):
        n = random.randint( low, high )
        if isprime(n):
            return n
    print( 'no prime' )
    return None

In [52]:
[ pickprime() for i in range(10) ]

[293, 619, 443, 733, 853, 521, 991, 547, 139, 773]

##  RSA

### 1. on choisit deux nombres premiers p = 3, q = 11 

In [53]:
p = pickprime()
q = pickprime()

print( "p=%i; q=%i"% (p, q) )

p=631; q=293


In [54]:
## avec l'exemple :
#p, q = 3, 11
#p, q = 61, 53

### 2. leur produit n = 3 × 11 = 33 est le module de chiffrement 

In [55]:
n = p*q

print("n=%i"%n)

n=184883


### 3. φ(n) = (3 – 1) × (11 – 1) = 2 × 10 = 20 ;

C'est la valeur de l'indicatrice d'Euler en n...  
C'est-à-dire le nombre d'entier inférieure à n, et premier avec n... ok..

In [56]:
phi_n = (p-1)*(q-1)

print("φ(n) = %i"%phi_n)

φ(n) = 183960


**Remarque:** _"Premier avec"_ signifie  plus grand diviseur commum égale à 1

#### Algorithme d'Euclide
Touve le plus grand commun diviseur (pgcd).  
Voir encore [wikipedia](https://fr.wikipedia.org/wiki/Algorithme_d%27Euclide)

In [57]:
def pgcd( a, b ):
    
    if b==0:
        return a
    elif b <= a:
        return pgcd( b, a%b )
    else:
        return pgcd( a, b%a )

def isprimewith( a,b ):
    if pgcd( a, b ) == 1:
        return True
    else:
        return False

In [58]:
pgcd( 60, 18 )

6

Est-ce que $\phi(n)$ est bien le nombre d'entier inférieure à n et premier avec n ?? 

In [59]:
def my_phi( n ):
    primewithn = []
    for i in range(1, n+1):
        if isprimewith( i, n ):
            primewithn.append( i )
            
    return primewithn

In [60]:
print( "my_phi: %i" % len( my_phi( n ) ) )
print("φ(n) = %i"%phi_n)

my_phi: 183960
φ(n) = 183960


magique... 

### 4. choisir un entier naturel e premier avec φ(n) et strictement inférieur à φ(n)

Il est appelé l'exposant de chiffrement

In [61]:
quandidats = []
for e in range(2, phi_n):
    if isprimewith(e, phi_n):
        quandidats.append(e)
    

In [62]:
e = random.choice( quandidats )
print( e )

8927


In [63]:
quandidats[:10]

[11, 13, 17, 19, 23, 29, 31, 37, 41, 43]

In [64]:
# On choisit e= 3 (premier avec 20) comme exposant de chiffrement ;

#e = 3
#e = 43

### 5. calculer l'entier naturel d, inverse de e modulo φ(n), et strictement inférieur à φ(n)

appelé exposant de déchiffrement ; d peut se calculer efficacement par l'algorithme d'Euclide étendu ([wikipedia](https://fr.wikipedia.org/wiki/Algorithme_d%27Euclide_%C3%A9tendu)).

On veut donc :  ``` e*d % phi(n) == 1 ```

ce qui veut dire qu'il y a un entier relatif k tel que :  ``` e*d + k*phi == 1 ```

In [65]:
a, b =   e, phi_n

# Entrée : a, b entiers (naturels)
# Sortie : r entier (naturel) et  u, v entiers relatifs tels que r = pgcd(a, b) et r = a*u + b*v

r, u, v, rp, up, vp = a, 1, 0, b, 0, 1

#les égalités r = a*u+b*v et r' = a*u'+b*v' sont des invariants de boucle

while rp !=0:
    q = int( r / rp ) 
    r, u, v, rp, up, vp = rp, up, vp, r - q *rp, u - q*up, v - q*vp

print(r, u, v)

# donc:

k = v
d = u

1 -577 28


In [66]:
# l'exposant de déchiffrement est d = 7, l'inverse de 3 modulo 20 (en effet ed = 3 × 7 ≡ 1 mod 20).

In [67]:
print( "d=%i"%d )

d=-577


In [68]:
e*d  % phi_n

1

In [69]:
e*d + k*phi_n

1

ok, mais d est négatif ??  
https://crypto.stackexchange.com/questions/10805/how-does-one-deal-with-a-negative-d-in-rsa

In [70]:
while d < 0 :
    d += phi_n

In [71]:
e*d  % phi_n

1

L'équation est toujours vérifiée 

In [72]:
print( "d=%i"%d )

d=183383


Le couple **(n,e)** est la clé publique du chiffrement,  
alors que le nombre **d** est sa clé privée,  


## Chiffrement du message

Si M est un entier naturel strictement inférieur à n représentant un message, alors le message chiffré sera représenté par

In [73]:
n

184883

In [74]:
M = 123456

In [75]:
C = M**e % n

In [76]:
print( C  )

41459


## Déchiffrement du message

In [77]:
M_dechif = C**d % n

In [78]:
print( M_dechif )

123456


yes...


* Comment faire pour caser le code... ?  factoriser n pour retrouver p et q ...
* Pourquoi $ (M^e \mod n)^d \mod n  = M       $

In [79]:
a = 451
p = 17
n = 31

print(  (a % n)**p )
print(  (a**p % n) )

827240261886336764177
21


In [82]:
C**d

...